<a href="https://colab.research.google.com/github/drlee1218/differential_privacy/blob/main/Step3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow

In [3]:
# Install required libraries
!pip install tensorflow-federated



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 77.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you w

In [1]:
!pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.9 MB/s eta 0:00:00


In [2]:
!pip install numpy
!pip install typing-extensions
!pip install scipy

In [3]:
import kagglehub

print("Downloading FER2013 dataset using kagglehub...")
path = kagglehub.dataset_download("msambare/fer2013")
print("Dataset downloaded to:", path)

# ---------------- Library Imports ----------------
import os
import time
import numpy as np
from glob import glob
from PIL import Image
import tensorflow as tf
import tensorflow_federated as tff
import tenseal as ts
from sklearn.model_selection import train_test_split

# Enable TensorFlow debugging
tf.debugging.set_log_device_placement(True)

# Config
IMG_SIZE, NUM_CLASSES, NUM_CLIENTS = 48, 7, 2
BATCH_SIZE, EPOCHS, ROUNDS = 16, 1, 3  # Increased batch size
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Comment out to enable GPU

# ---------------- Dataset Load ----------------
def load_images_from_folder(folder, label):
    images, labels = [], []
    for img_path in glob(os.path.join(folder, '*.jpg')):
        try:
            img = Image.open(img_path).convert('L').resize((IMG_SIZE, IMG_SIZE))
            images.append(np.array(img))
            labels.append(label)
        except:
            continue
    return np.array(images), np.array(labels)

def load_dataset(base_path, fraction=0.1):  # Use 10% of data
    X, y = [], []
    for idx, emo in enumerate(EMOTIONS):
        folder = os.path.join(base_path, emo)
        if not os.path.exists(folder):
            print(f"Warning: {folder} not found")
            continue
        imgs, lbls = load_images_from_folder(folder, idx)
        print(f"Loaded {len(imgs)} images for {emo}")
        num_samples = int(len(imgs) * fraction)
        indices = np.random.choice(len(imgs), num_samples, replace=False)
        X.append(imgs[indices])
        y.append(lbls[indices])
    if not X:
        print("Error: No images loaded")
        return np.array([]), np.array([])
    X = np.concatenate(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
    y = tf.keras.utils.to_categorical(np.concatenate(y), NUM_CLASSES)
    print(f"Dataset: X shape={X.shape}, y shape={y.shape}")
    return X, y

# ---------------- Federated Prep ----------------
def create_federated_data(X, y):
    size = len(X) // NUM_CLIENTS
    client_data = [
        tf.data.Dataset.from_tensor_slices((X[i*size:(i+1)*size], y[i*size:(i+1)*size])).batch(BATCH_SIZE)
        for i in range(NUM_CLIENTS)
    ]
    for i, ds in enumerate(client_data):
        print(f"Client {i} dataset element_spec: {ds.element_spec}")
    return client_data

# ---------------- Model ----------------
def create_keras_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(2, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),  # Further reduced
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(4, (3,3), activation='relu'),  # Further reduced
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8, activation='relu'),  # Further reduced
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    print("Model summary:")
    model.summary()
    return model

def model_fn():
    return tff.learning.models.from_keras_model(
        create_keras_model(),
        input_spec=client_data[0].element_spec,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()]
    )

# ---------------- TenSEAL ----------------
def initialize_tenseal_context():
    try:
        ctx = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=16384,
            coeff_mod_bit_sizes=[50, 30, 50]  # Simplified
        )
        ctx.global_scale = 2 ** 30  # Reduced scale
        ctx.generate_galois_keys()
        return ctx
    except Exception as e:
        print(f"Error initializing TenSEAL: {e}")
        exit()

def encrypt_array(ctx, array):
    try:
        print(f"Encrypting array of size {array.size}")
        return ts.ckks_vector(ctx, array.tolist())
    except Exception as e:
        print(f"Error encrypting array: {e}")
        return None

def decrypt_array(enc_array):
    try:
        result = np.array(enc_array.decrypt())
        print(f"Decrypted array shape: {result.shape}")
        return result
    except Exception as e:
        print(f"Error decrypting array: {e}")
        return None

# ---------------- Client Update ----------------
def client_update_fn(model_weights, dataset, ctx):
    start_time = time.time()
    model = create_keras_model()
    model.set_weights(model_weights)
    opt = tf.keras.optimizers.SGD(0.01)

    for x_batch, y_batch in dataset:
        with tf.GradientTape() as tape:
            preds = model(x_batch)
            loss = tf.keras.losses.CategoricalCrossentropy()(y_batch, preds)
        grads = tape.gradient(loss, model.trainable_weights)
        opt.apply_gradients(zip(grads, model.trainable_weights))

    new_weights = model.get_weights()
    delta = [nw - ow for nw, ow in zip(new_weights, model_weights)]
    enc_delta = [encrypt_array(ctx, d.flatten()) for d in delta]
    if None in enc_delta:
        print("Error: Failed to encrypt deltas")
        return None
    print(f"Client update took {time.time() - start_time:.2f} sec")
    return enc_delta

# ---------------- Server Update ----------------
def server_update_fn(encrypted_deltas, ctx, shapes, model_weights):
    start_time = time.time()
    agg = []
    print(f"Processing {len(encrypted_deltas)} encrypted deltas")
    for i in range(len(encrypted_deltas[0])):
        s = encrypted_deltas[0][i]
        for d in encrypted_deltas[1:]:
            s += d[i]
        s *= (1.0 / len(encrypted_deltas))
        agg.append(s)

    dec_deltas = [decrypt_array(e).reshape(shape) for e, shape in zip(agg, shapes)]
    if any(d is None for d in dec_deltas):  # Fixed ValueError
        print("Error: Failed to decrypt deltas")
        return model_weights
    new_weights = [w + d for w, d in zip(model_weights, dec_deltas)]
    print(f"Server update took {time.time() - start_time:.2f} sec")
    return new_weights

# ---------------- Training ----------------
dataset_path = os.path.join(path, "train")
X, y = load_dataset(dataset_path, fraction=0.1)
if len(X) == 0 or len(y) == 0:
    print("Error: Empty dataset")
    exit()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
client_data = create_federated_data(X_train, y_train)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)
print(f"Test dataset element_spec: {test_ds.element_spec}")

for x_batch, y_batch in test_ds.take(1):
    print(f"Sample batch: X shape={x_batch.shape}, y shape={y_batch.shape}")

# Init
model = create_keras_model()
weights = model.get_weights()
shapes = [w.shape for w in weights]
tenseal_ctx = initialize_tenseal_context()

print("Homomorphic Federated Training Start")
start = time.time()

for rnd in range(1, ROUNDS + 1):
    print(f"\nRound {rnd} starts")
    round_start = time.time()
    encrypted_deltas = []
    for i, cdata in enumerate(client_data):
        print(f"Processing client {i}")
        enc_delta = client_update_fn(weights, cdata, tenseal_ctx)
        if enc_delta is None:
            print(f"Round {rnd}: Client {i} update failed")
            continue
        encrypted_deltas.append(enc_delta)

    weights = server_update_fn(encrypted_deltas, tenseal_ctx, shapes, weights)
    model.set_weights(weights)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    try:
        loss, acc = model.evaluate(test_ds, verbose=0)
        print(f"Round {rnd}: Test Accuracy={acc:.4f}, Loss={loss:.4f}")
    except Exception as e:
        print(f"Round {rnd}: Test evaluation failed: {e}")
    print(f"Round {rnd} took {time.time() - round_start:.2f} sec")

end = time.time()
print(f"\nFinal Accuracy: {acc:.4f}")
print(f"Total Time: {end - start:.2f} sec")

📥 Downloading FER2013 dataset using kagglehub...
✅ Dataset downloaded to: /kaggle/input/fer2013


ERROR:jax._src.xla_bridge:Jax plugin configuration error: Plugin module %s could not be loaded
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/xla_bridge.py", line 428, in discover_pjrt_plugins
    plugin_module = importlib.import_module(plugin_module_name)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_r

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Executing op Conv2DBackpropInput in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Conv2DBackpropFilter in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Neg in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul